# The Diet problem
(Adapted from the pyomo/examples/pyomo files and Ch2 of the ampl book)

> **Requirements**
> - python3.X
> - pyomo 5.X.X
> - [cbc](https://github.com/coin-or/Cbc) 2.10 (tested with)

### 0. Introduction

The goal of this problem is to *minimize* the cost of a meal given several available food items. The variables of the problem model the decision of weather a certain food is bought or not.  

\begin{align}
\text{minimize} \quad & \sum_{j \in foods} C_{j} x_{j} \\
\text{subject to} \quad & \sum_{j \in \left\lbrace meals, sides, drinks \right\rbrace} x_{j} \geq 1\\
&  0 \leq x_{j} \leq M, \forall j \in foods
\end{align}

Furthermore, the constraints ensure that *at least* one entree, side and drink are selected. Variations of this problem include constraints for certain nutritional requirements (e.g. vitamins) to be met ($\sum_{j \in foods} p_{ij} x_{j} \geq l_{i}, \forall i \in N$).

This model is written with the following structure:  

1. Package imports
2. Model and Set declaration
3. Parameter declaration and initialization
4. Variable declaration and initialization
5. Constraint declaration and initialization
6. Objective function
7. Solution

### 1. Package imports

We need to import certain packages to the current workspace, i.e. pyomo components.

In [34]:
from pyomo.environ import *   #: This imports most of the usual pyomo objects
from pyomo.opt import SolverFactory  #: This imports the solver modules

### 2. Model and Set declaration

Note how the model *must* be declared first. Subsequently, every component from the model is attached to the model.

In [2]:
m = ConcreteModel()  #: Declare model

The set *food* contains the overall available dishes.  
In order to create this set, there are a number of ways to specify its items.   
*One* of the possible ways is to specify its items as a *python list* of strings, i.e.:

In [3]:
foods = ["QPwCheese", "MDwCheese", "LeBigMac", "FOFish", "McGChicken", "Fries", "McSausage", "LfMilk", "OJ"]

Then, the set is *initialized* using the `initialize` keyword of the Set constructor,

In [35]:
m.food = Set(initialize=foods)
m.food.pprint()

    'pyomo.core.base.sets.SimpleSet'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.sets.SimpleSet'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
food : Dim=0, Dimen=1, Size=9, Domain=None, Ordered=False, Bounds=None
    ['FOFish', 'Fries', 'LeBigMac', 'LfMilk', 'MDwCheese', 'McGChicken', 'McSausage', 'OJ', 'QPwCheese']


Finally, we can reuse the dictionary to define the kinds of dishes available,

In [36]:
entree = foods[:5]
side = foods[5:7]
drink = foods[7:]
print("entres: ", entree)
print("sides: ", side)
print("drinks: ", drink)

entres:  ['QPwCheese', 'MDwCheese', 'LeBigMac', 'FOFish', 'McGChicken']
sides:  ['Fries', 'McSausage']
drinks:  ['LfMilk', 'OJ']


### 3. Parameter declaration and initialization

The parameters are declared over a set (e.g. food). To specify the values of such parameters, it is possible to use a dictionary whose key correspond to the values of the set.

In [6]:
COST_DICT = dict.fromkeys(foods)
COST_DICT = {"QPwCheese": 1.84,
             "MDwCheese": 2.19, 
             "LeBigMac": 1.84, 
             "FOFish": 1.44, 
             "McGChicken": 2.29, 
             "Fries": 0.77, 
             "McSausage": 1.29, 
             "LfMilk": 0.60, 
             "OJ": 0.72}

Then, the dictionary is passed to the respective parameter using the `initialize` keyword.

In [37]:
m.cost = Param(m.food, initialize=COST_DICT, within=PositiveReals)
m.cost.display()

    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
cost : Size=9, Index=food, Domain=PositiveReals, Default=None, Mutable=False
    Key        : Value
        FOFish :  1.44
         Fries :  0.77
      LeBigMac :  1.84
        LfMilk :   0.6
     MDwCheese :  2.19
    McGChicken :  2.29
     McSausage :  1.29
            OJ :  0.72
     QPwCheese :  1.84


Another way to initialize parameters over a set is using the `default` keyword.

In [8]:
m.f_min = Param(m.food, within=NonNegativeReals, default=0.0)

In [38]:
MAX_FOOD_SUPPLY = 20
print(MAX_FOOD_SUPPLY)

20.0


In [10]:
m.f_max = Param(m.food, default=MAX_FOOD_SUPPLY)

A third way is to use a python function, and setting the keyword to the function name (`initialize=function_name`).

### 4. Variables

As a fundamental part of the model, variables have several properties; like bounds and domain. Most of these properties can be set when the variable is constructed.  
In a similar fashion as before, the *keywords* arguments can be used for this purpose. In the following example, the variable *buy* is constructed with specific bounds given by a python function `buy_bounds`,

In [11]:
def buy_bounds(mod, i):  #: declare the function first
    return (mod.f_min[i], mod.f_max[i])

Then pass the function to the variable!.

In [12]:
m.buy = Var(m.food, bounds=buy_bounds, within=NonNegativeIntegers)

Note that the python interpreter will read the script from top to bottom. Therefore the function(s) used to construct the variable (or objects, e.g. constraint, set, parameter) must be declared *before* they are referenced.

<div class="alert alert-block alert-info">
<b>Alert:</b> The function `buy_bounds` has an argument `mod`. `mod` and `m` are not the same, until it gets passed to the `Var` constructor.
</div>

### 5. Constraints

In pyomo, in order to construct a constraint, it is necessary to provide an *expression*. In other words, a constraint *contains* an expression, e.g. an equality. For an indexed constraint, an expression is required for *every* element of the set.  
The most common way of assigning an expressions for single or indexed constraints is through a python function. e.g.

In [13]:
def entree_rule(mod):  # note that mod is an argument!, mod and m are not necessarily the same
    return sum(mod.buy[e] for e in entree) >= 1

This function will *return* an expression; namely `sum(mod.buy[e] for e in entree) >= 1`. Then, the at the construction of the constraint, the keyword `rule` is intended to take a function that will provide the expression(s) for the constraint, 

In [14]:
m.entree = Constraint(rule=entree_rule)

In [39]:
m.entree.pprint()

entree : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                            : Upper : Active
    None :   1.0 : buy[QPwCheese] + buy[MDwCheese] + buy[LeBigMac] + buy[FOFish] + buy[McGChicken] :  +Inf :   True


This can be used for most constraints within a model.

In [15]:
def side_rule(mod):  # note that mod and m are not necessarily equal!
    return sum(mod.buy[s] for s in side) >= 1

Then pass the `side_rule` to the constructor.

In [16]:
m.side = Constraint(rule=side_rule)

Moreover, if the constraint is a *single* constraint; the expression can be declared directly,

In [17]:
m.drink = Constraint(expr=sum(m.buy[d] for d in drink) >=1)

Where the `expr` keyword is used for this purpose.

### 6. Objective

While several objectives can be declared, it is typical to have a single expression as part of the objective and a *sense*, i.e. minimize or maximize.  
The objective in this problem is to minimize the cost of the meal, i.e. $\sum_{i \in foods} C_{i} x_{i}$, where $C_{i}$ is the cost of item $i$ and $X_{i}$ is the number of items $i$ purchased.  
In the following example a python function is used to return the expression and then the keyword `rule` is used during the construction of the Objective.

In [18]:
def total_cost_rule(mod):
    return sum(mod.cost[j] * mod.buy[j] for j in mod.food)
m.total_cost = Objective(rule=total_cost_rule, sense=minimize)

### 7. Solution

The solution of the problem can be done in a number of ways. Often, users would solve the problem *ad hoc*. In other words, solve within the script where the model has been declared.  
Other ways of solving these problems include the command `pyomo solve challenging_model.py --solver=my_favourite_solver` in the terminal.  
For now, the previously declared model `m` will be solved *ad hoc*. This requires calling a solver, which is part of a solver object of the class `SolverFactory`.  
Firstly, assuming that the desired solver is in the *PATH*, declare the solver object:

In [41]:
opt = SolverFactory('ipopt')  #: declare the solver note: you can use ipopt though the solution won't be correct

In [43]:
m.pprint()

3 Set Declarations
    amt_index : Dim=0, Dimen=2, Size=63, Domain=None, Ordered=False, Bounds=None
        Virtual
    food : Dim=0, Dimen=1, Size=9, Domain=None, Ordered=False, Bounds=None
        ['FOFish', 'Fries', 'LeBigMac', 'LfMilk', 'MDwCheese', 'McGChicken', 'McSausage', 'OJ', 'QPwCheese']
    nutr : Dim=0, Dimen=1, Size=7, Domain=None, Ordered=False, Bounds=None
        ['Cal', 'Calc', 'Carbo', 'Iron', 'Protein', 'VitA', 'VitC']

6 Param Declarations
    amt : Size=63, Index=amt_index, Domain=Any, Default=None, Mutable=False
        Key                       : Value
                ('FOFish', 'Cal') :   370
               ('FOFish', 'Calc') :    15
              ('FOFish', 'Carbo') :    38
               ('FOFish', 'Iron') :    10
            ('FOFish', 'Protein') :    14
               ('FOFish', 'VitA') :     2
               ('FOFish', 'VitC') :     0
                 ('Fries', 'Cal') :   220
                ('Fries', 'Calc') :     0
               ('Fries', 'Carbo') :    

Then, call the solver to solve the problem given by model `m`,

In [42]:
results = opt.solve(m, tee=True)

Ipopt 3.11.1: 
==> Warning: Treating 0 binary and 9 integer variables as continous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       67
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        9
                     variables with only lower bounds:    

If the solver was *successful*. We can extract the results directly from the variables of the problem (e.g. using the `value(var)` build-in function). In this case the contents of the variable `buy` are displayed directly.

In [21]:
m.buy.display()  #: display contents

buy : Size=9, Index=food
    Key        : Lower : Value : Upper : Fixed : Stale : Domain
        FOFish :   0.0 :   1.0 :  20.0 : False : False : NonNegativeIntegers
         Fries :   0.0 :   1.0 :  20.0 : False : False : NonNegativeIntegers
      LeBigMac :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
        LfMilk :   0.0 :   1.0 :  20.0 : False : False : NonNegativeIntegers
     MDwCheese :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
    McGChicken :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
     McSausage :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
            OJ :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
     QPwCheese :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers


### 7. Conclusions

The diet problem can be *conveniently* constructed using basic python data-structures and functions. This particular example can be constructed in several ways. For example using `AbstractModel` as it is used in the pyomo documentation.

### Optional material

A similar problem can consider purchasing a set of items satisfying certain nutritional requirements. For this a new constraint has to be considered with some extra data.

In [22]:
NUTR = ["Cal", "Carbo", "Protein", "VitA", "VitC", "Calc", "Iron"]

In [23]:
m.nutr = Set(initialize=NUTR)  #: New set of nutrients

In [24]:
N_MIN = {"Cal": 2000, "Carbo": 350, "Protein": 55, "VitA": 100, "VitC": 100, "Calc": 100, "Iron": 100}

In [25]:
m.n_min = Param(m.nutr, initialize=N_MIN)  #: minimum amounts of nutrients

In [26]:
N_MAX = {"Cal": float('inf'), "Carbo": 375, "Protein": float('inf'), "VitA": float('inf'), "VitC": float('inf'), "Calc": float('inf'), "Iron": float('inf')}

In [27]:
m.n_max = Param(m.nutr, initialize=N_MAX)  #: maximum amounts of nutrients

In [28]:
AMT = {}
AMT[("QPwCheese","Cal")] = 510
AMT[("MDwCheese","Cal")] = 370
AMT[("LeBigMac","Cal")] = 500
AMT[("FOFish","Cal")] = 370
AMT[("McGChicken","Cal")] = 400
AMT[("Fries","Cal")] = 220
AMT[("McSausage","Cal")] = 345
AMT[("LfMilk","Cal")] = 110
AMT[("OJ","Cal")] = 80
AMT[("QPwCheese","Carbo")] = 34
AMT[("MDwCheese","Carbo")] = 35
AMT[("LeBigMac","Carbo")] = 42
AMT[("FOFish","Carbo")] = 38
AMT[("McGChicken","Carbo")] = 42
AMT[("Fries","Carbo")] = 26
AMT[("McSausage","Carbo")] = 27
AMT[("LfMilk","Carbo")] = 12
AMT[("OJ","Carbo")] = 20
AMT[("QPwCheese","Protein")] = 28
AMT[("MDwCheese","Protein")] = 24
AMT[("LeBigMac","Protein")] = 25
AMT[("FOFish","Protein")] = 14
AMT[("McGChicken","Protein")] = 31
AMT[("Fries","Protein")] = 3
AMT[("McSausage","Protein")] = 15
AMT[("LfMilk","Protein")] = 9
AMT[("OJ","Protein")] = 1
AMT[("QPwCheese","VitA")] = 15
AMT[("MDwCheese","VitA")] = 15
AMT[("LeBigMac","VitA")] = 6
AMT[("FOFish","VitA")] = 2
AMT[("McGChicken","VitA")] = 8
AMT[("Fries","VitA")] = 0
AMT[("McSausage","VitA")] = 4
AMT[("LfMilk","VitA")] = 10
AMT[("OJ","VitA")] = 2
AMT[("QPwCheese","VitC")] = 6
AMT[("MDwCheese","VitC")] = 10
AMT[("LeBigMac","VitC")] = 2
AMT[("FOFish","VitC")] = 0
AMT[("McGChicken","VitC")] = 15
AMT[("Fries","VitC")] = 15
AMT[("McSausage","VitC")] = 0
AMT[("LfMilk","VitC")] = 4
AMT[("OJ","VitC")] = 120
AMT[("QPwCheese","Calc")] = 30
AMT[("MDwCheese","Calc")] = 20
AMT[("LeBigMac","Calc")] = 25
AMT[("FOFish","Calc")] = 15
AMT[("McGChicken","Calc")] = 15
AMT[("Fries","Calc")] = 0
AMT[("McSausage","Calc")] = 20
AMT[("LfMilk","Calc")] = 30
AMT[("OJ","Calc")] = 2
AMT[("QPwCheese","Iron")] = 20
AMT[("MDwCheese","Iron")] = 20
AMT[("LeBigMac","Iron")] = 20
AMT[("FOFish","Iron")] = 10
AMT[("McGChicken","Iron")] = 8
AMT[("Fries","Iron")] = 2
AMT[("McSausage","Iron")] = 15
AMT[("LfMilk","Iron")] = 0
AMT[("OJ","Iron")] = 2

In [29]:
m.amt = Param(m.food, m.nutr, initialize=AMT)  #: nutrition information

With all the nutritional contents, the constraint can be created. For this a double sided inequality expression is used:

In [30]:
def nutr_constraint_init(mod, j):
    return inequality(mod.n_min[j], sum(mod.amt[i, j] * mod.buy[i] for i in mod.food), mod.n_max[j])  #: new: use inequality() for chained inequalities :S

Note the use of the `sum` built-in function. The `nutr_constraint_init` function basically returns an expression in the form $n_{min,j} \leq \sum_{i\in F} a_{i,j} x_{i} \leq n_{max, j}$ This function is then passed to the `rule` keyword in the constructor.

In [31]:
m.nutr_con = Constraint(m.nutr, rule=nutr_constraint_init)

And then solve again.

In [32]:
res = opt.solve(m, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Nov 24 2015 

command line - C:\Users\dav0\cbc-win64\cbc.exe -printingOptions all -import C:\Users\dav0\AppData\Local\Temp\tmpve9nf8uy.pyomo.lp -stat=1 -solve -solu C:\Users\dav0\AppData\Local\Temp\tmpve9nf8uy.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 11 (-1) rows, 9 (-1) columns and 76 (-1) elements
Statistics for presolved model
Original problem has 9 integers (0 of which binary)
Presolved problem has 9 integers (0 of which binary)
==== 0 zero objective 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objective of 0.77
1 variables have objective of 1.29
1 variables have objective of 1.44
2 variables have objective of 1.84
1 variables have objective of 2.19
1 variables have objective of 2.29
==== absolute objective values 8 different
1 variables have objective of 0.6
1 variables have objective of 0.72
1 variables have objecti

In [33]:
m.buy.pprint()   #: I hope you like fries.

buy : Size=9, Index=food
    Key        : Lower : Value : Upper : Fixed : Stale : Domain
        FOFish :   0.0 :   1.0 :  20.0 : False : False : NonNegativeIntegers
         Fries :   0.0 :   5.0 :  20.0 : False : False : NonNegativeIntegers
      LeBigMac :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
        LfMilk :   0.0 :   4.0 :  20.0 : False : False : NonNegativeIntegers
     MDwCheese :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
    McGChicken :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
     McSausage :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
            OJ :   0.0 :   0.0 :  20.0 : False : False : NonNegativeIntegers
     QPwCheese :   0.0 :   4.0 :  20.0 : False : False : NonNegativeIntegers


### Credits:
- David Thierry (Carnegie Mellon University 2019)